**Basic Analysis**
* Number of user queries
* Number of total characters (unicode?)
    * Number of *human languages* among all conversations
* Number of *Tools*
    * Number of Multimedia
        * from: users/assistants
        * type: image/file/\<code\>
* Size of HTML

**IDEA**
* If the conversation is to query some concept, or the goal is to solve some practical problems. Use the conversation queries to make google/bing search: paraphrase the query to see if there is an "ideal query"? (SIGIR survey: improve with today's SE unchanged!)
* How many Searches are done per query? What are the queries? Did LLMs perform search with semantic understanding?

**More conversational LLMs?** Chatgpt is known to spit too much text.

**Other Implications**

Other things to measure? People switching between models? I don't know what this means in the underlying system...

This is by nature dataset of LLM generated language in a conversation...

In [1]:
import os
import json

In [2]:
conversation_fnames = os.listdir('conversations')
conversation_fnames = [fname for fname in conversation_fnames if '.json' in fname]

In [13]:
conversation_example = conversation_fnames[0]
with open(f'conversations/{conversation_example}', 'r') as f:
    conversation = json.load(f)

conversation

{'title': 'Carlos Nogueira Art Overview',
 'conversation': [{'role': 'system', 'message': [''], 'text': ''},
  {'role': 'user',
   'message': ['Original custom instructions no longer available'],
   'text': ''},
  {'role': 'user', 'message': ['Who is Carlos Nogueira?'], 'text': ''},
  {'role': 'assistant', 'message': [], 'text': ''},
  {'role': 'assistant', 'message': [], 'text': 'search("Carlos Nogueira")'},
  {'role': 'tool', 'message': [], 'text': ''},
  {'role': 'assistant', 'message': [], 'text': 'mclick([0, 2, 3, 4])'},
  {'role': 'tool',
   'message': [],
   'text': '\n— Alcoutim, linha de luz (projeto de instalação de luz para uma proposta de arquitetura de José Adrião | project for a light installation for an architectural proposal of José Adrião)\n\n2003  \n— Lisboa | Lisbon, Galeria Promontório Arquitetos, desenhos de casas, 18 de setembro – 3 de novembro | September 18th – November 3rd\n\n— Lisboa | Lisbon, Galeria Luís Serpa Projetos, construção para lugar nenhum, 20 de se

In [15]:
"""
Count the number of backNforth in a conversation
"""

roles = ['user', 'assistant']
backNforth = 0
prev_role = 'user'
for take in conversation['conversation']:
    if take['role'] == 'user':
        prev_role = 'user'
    elif take['role'] == 'assistant' and prev_role == 'user':
        prev_role = 'assistant'
        backNforth += 1

backNforth

2

In [16]:
# find the primary language of the max=5 queries

queries = [take['message'] for take in conversation['conversation']if take['role']=='user']
queries.pop(0)

from langdetect import detect, detect_langs

langset = set()
for i in range(min(len(queries), 5)):
    lang = detect(queries[i][0])
    langset.add(lang)

list(langset)

['en']

In [17]:
# count of characters
#   this also includes code
#   this includes markdown language... so it's not good

user_messages = queries
assistant_messages = [take['message'] for take in conversation['conversation']if take['role']=='assistant']

charcount_user = sum([len(take[0]) for take in user_messages if len(take)!=0])
charcount_assistant = sum([len(take[0]) for take in assistant_messages if len(take)!=0])

charcount_user, charcount_assistant

(166, 3299)

In [31]:
# TODO: match user queries with search() by assistant
match_search = {}
k = None
for take in conversation['conversation']:
    if take['role'] == 'user':
        k = take.get('message', [''])
        k = k[0]
    if k and take['role'] != 'user':
        message = take.get('message', [])
        text = take.get('text', '')
        if k not in match_search:
            match_search[k] = []
        if message:
            match_search[k].append(message[0])
        if text:
            match_search[k].append(text)

search_agg = {}
for k, v in match_search.items():
    search_agg[k] = []
    for text in v:
        if "search(" in text:
            search_agg[k].append(text)

print(json.dumps(search_agg, indent=4))

{
    "Who is Carlos Nogueira?": [
        "search(\"Carlos Nogueira\")"
    ],
    "This is not the Carlos Nogueira I'm looking for. He has been working in artificial intelligence, developing solutions for customers like Serasa": [
        "search(\"Carlos Nogueira artificial intelligence Serasa\")"
    ]
}


total #conv 1800

#conv w search() 180

#search() 288

In [18]:
# character count for code OR structured data

assistant_messages = [take['message'] for take in conversation['conversation']if take['role']=='assistant']

import re
texts = []
for take in assistant_messages:
    texts += take

for text in texts:
    code_blocks = re.findall(r"```(.*?)```", text, re.DOTALL)
    # Print extracted code blocks
    print([{
        'type': code_block.split('\n')[0].strip(),
        'length': len('\n'.join(code_block.split('\n')[1:]))
    } for code_block in code_blocks])

    # for code_block in code_blocks:
    #     code_type = code_block.split('\n')[0].strip()
    #     code_length = len(code_block)
    # for i, code in enumerate(code_blocks, 1):
    #     print(f"Code Block {i}:\n{code}\n")
    

[]
[]


In [9]:
# Tools... not finished yet... are we interested in this? 
#   we would have no access to the files anyways, and we can ask gpt to generate this...
#   all previous measurements focus on counting, which GPT is not super good at, especially at counting workds

# for i in range(10):
i=3
conversation_example = conversation_fnames[i]
with open(f'conversations/{conversation_example}', 'r') as f:
    conversation = json.load(f)

print(conversation['url'])
tool_messages = [take['message'] for take in conversation['conversation']if take['role']=='tool']
print(tool_messages)
conversation['conversation']

KeyError: 'url'